<h1>CNN-H1 4-22-2016</h1>

<strong>Abstract</strong>
Implementing the CNN-H1 using NN2 described in the paper: http://arxiv.org/pdf/1509.00244v1.pdf. 

<strong>Implementation</strong>
<ul>
<li>Normalizes images to 230x230 centered around the facial features and then takes the middle 165X120 portion</li>
<li>Validation split of 15%</li>
<li>fc6 layer has a length of 512</li>
<li>Noise with width 15</li>
<li>Number of people is 600</li>
</ul>

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
%matplotlib inline

import dlib
import os
import fnmatch
import math
import numpy as np
import matplotlib.pyplot as plt
import sys

from matplotlib.pyplot import imshow
from scipy.ndimage.filters import gaussian_filter
from scipy.spatial import distance
from scipy.misc import imrotate

from skimage import io
from skimage.color import rgb2grey
from skimage.transform import resize

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from keras.models import model_from_json

np.random.seed(123456)

Using Theano backend.
Using gpu device 0: GRID K520 (CNMeM is enabled with initial size: 98.0% of memory, CuDNN 3007)


In [3]:
data_lfw_path = '../data/lfw'
data_path = '../data'
norm_shape = 230, 230
input_shape = 165, 120
noise_width = 15
num_people = 600

<h2>Loading Files</h2>

In [4]:
def get_face_to_file_path_dict():
    face_to_file_paths_dict = {}
    
    for root, dirnames, filenames in os.walk(data_lfw_path):
        for dirname in dirnames:
            if dirname not in face_to_file_paths_dict:
                face_to_file_paths_dict[dirname] = []
            directory_path = os.path.join(data_lfw_path, dirname)
            for filename in os.listdir(directory_path):
                if filename.endswith(".jpg"):
                    face_to_file_paths_dict[dirname].append(os.path.join(directory_path, filename))
                            
    return face_to_file_paths_dict

In [5]:
def get_face_to_file_paths_descending_list(face_to_file_paths_dict):
    return sorted(face_to_file_paths_dict.items(), key=lambda x: len(x[1]), reverse=True)

In [6]:
face_to_file_paths_dict = get_face_to_file_path_dict()

In [7]:
face_to_file_paths_list = get_face_to_file_paths_descending_list(face_to_file_paths_dict)[:num_people]

<h2>Helper Functions</h2>

In [8]:
def get_centroid(points):
    x = [p[0] for p in points]
    y = [p[1] for p in points]
    return (sum(x) / len(points), sum(y) / len(points))
    
def get_facial_feature_points(im, face):
    points = np.array([[p.x, p.y] for p in predictor(im, face).parts()])
    facial_feature_points = [None for i in range(5)]
    
    # Left and right eyes, nose, left and right points of the mouth
    facial_feature_points[0] = get_centroid(np.concatenate((points[37:39], points[40:42])))
    facial_feature_points[1] = get_centroid(np.concatenate((points[43:45], points[46:48])))
    facial_feature_points[2] = points[30]
    facial_feature_points[3] = points[48]
    facial_feature_points[4] = points[54]
    
    return np.array(facial_feature_points)

def get_facial_landmark_points(im, face, w=.6, eye_scale=.05):
    points = np.array([[p.x, p.y] for p in predictor(im, face).parts()])
    landmarks = [None for _ in range(9)]
    
    p = get_centroid([points[19], points[24]])
    dx = p[0] - points[51][0]
    dy = p[1] - points[51][1]

    # Left
    landmarks[0] = points[19]
    
    p = get_centroid(points[40:42])
    landmarks[1] = (p[0] - dx * eye_scale, p[1] - dy * eye_scale)
    
    landmarks[2] = (points[2][0] * (1 - w) + w * points[31][0],
                      points[2][1] * (1 - w) + w * points[31][1])
    
    # Centre
    p = get_centroid([points[19], points[24]])
    landmarks[3] = (p[0] + dx * .1, p[1] + dy * .1)
    landmarks[4] = points[29]
    landmarks[5] = points[51]
    
    # Right
    landmarks[6] = points[24]
    
    p = get_centroid(points[46:48])
    landmarks[7] = (p[0] - dx * eye_scale, p[1] - dy * eye_scale)
    
    landmarks[8] = (points[14][0] * (1 - w) + w * points[35][0],
                      points[14][1] * (1 - w) + w * points[35][1])
    
    return np.array(landmarks)

In [9]:
def get_image_window(image, size, point):
    """
    Assume image is grey image
    """
    top = int(point[1] - size[0] / 2)
    left = int(point[0] - size[1] / 2)
    return image[top:top + size[0], left:left + size[1]]


def get_most_centre_face(image):
    height = image.shape[0]
    width = image.shape[1]
    image_centre = width / 2, height / 2
    most_centre_face = None
    most_centre_dist = sys.maxint
    
    detected_faces = detector(img, 1)
    for d in detected_faces:
        face_centre = (d.right() + d.left()) / 2, (d.bottom() + d.top()) / 2
        centre_dist = distance.euclidean(image_centre, face_centre)
        if centre_dist < most_centre_dist:
            most_centre_dist = centre_dist
            most_centre_face = d
            
    return most_centre_face

def calculate_rotation(points):
    """
    Return degrees of how much the face is rotated off the vertical axis
    """
    u = points[3]
    v = points[5]
    return math.degrees(np.arctan((v[0] - u[0]) / (v[1] - u[1])))

In [10]:
def reflection(image):
    return np.array([list(reversed(row)) for row in image])

def partition(image, top_left, rows, cols):
    return np.array([row[top_left[1]:top_left[1] + cols] for row in image[top_left[0]:top_left[0] + rows]])

In [11]:
def scale_coords(coords, resize_shape, original_shape):
    scale = np.array(resize_shape).astype(float) / np.array(original_shape)
    coords[:,0] = coords[:,0] * scale[1]
    coords[:,1] = coords[:,1] * scale[0]
    return coords

In [12]:
def get_random_noise_image(image, coords, width):
    """
    Apply random gaussian generated values
    and distribute them on gaussian distributed square
    centered on the coordinates passed in for the image
    """
    
    noise = np.zeros((image.shape[0], image.shape[1]))
    for coord in coords:
        # Convert coordinates to rows / columns
        apply_noise_at_point(coord[1], coord[0], noise, width)
    return np.clip(image + noise, 0, 1)

def apply_noise_at_point(x, y, noise, width):
    """
    Generate a block with a single random value placed at the center
    Apply the Gaussian filter with std of 4
    Place it on the noise array at the appropriate coordinates
    
    x represents the rows
    y represents the cols
    """
    
    block = np.zeros((width, width))
    block[width / 2, width / 2] = np.random.normal()
    block = gaussian_filter(block, sigma=4)
    
    x -= width / 2
    y -= width / 2
    
    x_end = min(noise.shape[0] - x, block.shape[0])
    x_start =  max(0, -x)

    y_end = min(noise.shape[1] - y, block.shape[1])
    y_start = max(0, -y)

    noise[max(0, x):x+block.shape[0], max(0, y):y+block.shape[1]] = block[x_start:x_end,y_start:y_end]

In [13]:
def apply_noise(image, coords):
    return get_random_noise_image(image, coords, noise_width)

<h2>Data Pre-Processing</h2>

In [14]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(os.path.join(data_path, 'shape_predictor_68_face_landmarks.dat'))

In [15]:
images_by_class = [[io.imread(f) for f in x[1]] for x in face_to_file_paths_list]

In [16]:
for i in range(len(images_by_class)):
    for j in range(len(images_by_class[i])):
        try:
            img = images_by_class[i][j]
            # Rotate image
            face = get_most_centre_face(img)
            landmark_points = get_facial_landmark_points(img, face)
            angle = calculate_rotation(landmark_points)
            img = imrotate(img, -angle)

            # Normalize and cropping image by centering around the facial features
            face = get_most_centre_face(img)
            landmark_points = get_facial_landmark_points(img, face)
            feature_points = get_facial_feature_points(img, face)

            face_height = face.bottom() - face.top()
            face_width = face.right() - face.left()
            scale_height = .5
            scale_width = .5

            left = max(0, face.left() - int(face_width * scale_width))
            top = max(0, face.top() - int(face_height * scale_height))
            right = min(img.shape[1], face.right() + int(face_width * scale_width))
            bottom = min(img.shape[0], face.bottom() + int(face_height * scale_height))

            img = img[top:bottom, left:right]
            landmark_points[:,0] -= left
            landmark_points[:,1] -= top
            feature_points[:,0] -= left
            feature_points[:,1] -= top

            # Resizing image and scaling facial landmark and feature points
            img = rgb2grey(img)            
            img_shape = img.shape
            img = resize(img, norm_shape)

            scale = np.array(norm_shape).astype(float) / np.array(img_shape)
            landmark_points[:,0] = landmark_points[:,0] * scale[1]
            landmark_points[:,1] = landmark_points[:,1] * scale[0]
            feature_points[:,0] = feature_points[:,0] * scale[1]
            feature_points[:,1] = feature_points[:,1] * scale[0]

            images_by_class[i][j] = (img, landmark_points, feature_points)
        except TypeError:
             images_by_class[i][j] = None

In [17]:
images_by_class = [[image for image in images if image is not None] for images in images_by_class]

In [18]:
# Create reflection with reflected coordinates
for i in range(len(images_by_class)):
    for j in range(len(images_by_class[i])):
        im, landmarks, features = images_by_class[i][j]
        new_features = [(im.shape[1] - p[0], p[1]) for p in features]
        new_landmarks = [(im.shape[1] - p[0], p[1]) for p in landmarks]
        new_landmarks[:3], new_landmarks[6:] = new_landmarks[6:], new_landmarks[:3]
        images_by_class[i].append((reflection(im), new_landmarks, new_features))

In [19]:
# Doubling the images and applying random gaussian noise
for i in range(len(images_by_class)):
    images_by_class[i] += images_by_class[i][:]
    images_by_class[i] = [(apply_noise(im, features), landmarks) 
                          for im, landmarks, features in images_by_class[i]]

In [20]:
# Get image of size 165x120, similar to the paper
for i in range(len(images_by_class)):
    images_by_class[i] = [get_image_window(im, input_shape, landmarks[4])
                          for im, landmarks in images_by_class[i]]

In [21]:
from collections import Counter
Counter([image.shape
            for images in images_by_class 
            for image in images if image.shape == input_shape
            ]).most_common(10)

[((165, 120), 26668)]

In [22]:
X_train = [image 
            for images in images_by_class 
            for image in images 
            if image.shape == input_shape
            ]
y_train = [images[0] 
            for images in enumerate(images_by_class) 
            for image in images[1] 
            if image.shape == input_shape
            ]

In [23]:
zipped = np.array(zip(X_train, y_train))
np.random.shuffle(zipped)

In [24]:
X_train = np.array([x[0] for x in zipped])
y_train = np.array([x[1] for x in zipped])

In [25]:
X_train = X_train.reshape(X_train.shape[0], 1, input_shape[0], input_shape[1])
Y_train = np_utils.to_categorical(y_train, len(images_by_class))

<h2> Training and Validation</h2>

In [26]:
def NN2(shape, nb_classes, nb_fc6):
    model = Sequential()
    
    # Layer 1
    model.add(Convolution2D(64, 3, 3, activation='relu', input_shape=shape))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    # Layer 2
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    # Layer 3
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    #Layer 4
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    #Layer 5
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3))
    model.add(ZeroPadding2D((1,1)))
    model.add(AveragePooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(nb_fc6))
    model.add(Dense(nb_classes, activation='softmax'))
   
    return model

In [27]:
model_input_shape = (1, input_shape[0], input_shape[1])
nb_fc6 = 512

In [28]:
model = NN2(model_input_shape, num_people, nb_fc6)
model.compile(loss='categorical_crossentropy', optimizer='sgd')
model.fit(X_train, Y_train, batch_size=32, nb_epoch=10, 
        show_accuracy=True, verbose=1, shuffle=True, validation_split=.15)

Train on 22667 samples, validate on 4001 samples
Epoch 1/10
22667/22667 [==============================] - 738s - loss: 5.8960 - acc: 0.0713 - val_loss: 5.7173 - val_acc: 0.0855
Epoch 2/10
22667/22667 [==============================] - 738s - loss: 5.7443 - acc: 0.0781 - val_loss: 5.6357 - val_acc: 0.0855
Epoch 3/10
22667/22667 [==============================] - 738s - loss: 5.4763 - acc: 0.0895 - val_loss: 5.1808 - val_acc: 0.1157
Epoch 4/10
22667/22667 [==============================] - 738s - loss: 4.8810 - acc: 0.1390 - val_loss: 4.7553 - val_acc: 0.1790
Epoch 5/10
22667/22667 [==============================] - 739s - loss: 3.9866 - acc: 0.2499 - val_loss: 3.4491 - val_acc: 0.3164
Epoch 6/10
22667/22667 [==============================] - 738s - loss: 2.6569 - acc: 0.4326 - val_loss: 2.3956 - val_acc: 0.4724
Epoch 7/10
22667/22667 [==============================] - 739s - loss: 1.4839 - acc: 0.6437 - val_loss: 1.7597 - val_acc: 0.5974
Epoch 8/10
22667/22667 [========================

In [29]:
json_string = model.to_json()
open('models/CNN-H1_epoch-10.json', 'w').write(json_string)
model.save_weights('models/CNN-H1_epoch-10.h5', overwrite=True)

In [30]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.001))
model.fit(X_train, Y_train, batch_size=32, nb_epoch=10,
        show_accuracy=True, verbose=1, shuffle=True, validation_split=.15)

Train on 22667 samples, validate on 4001 samples
Epoch 1/10
22667/22667 [==============================] - 739s - loss: 0.0538 - acc: 0.9846 - val_loss: 0.1868 - val_acc: 0.9630
Epoch 2/10
22667/22667 [==============================] - 739s - loss: 0.0241 - acc: 0.9944 - val_loss: 0.1838 - val_acc: 0.9640
Epoch 3/10
22667/22667 [==============================] - 739s - loss: 0.0155 - acc: 0.9961 - val_loss: 0.1915 - val_acc: 0.9663
Epoch 4/10
22667/22667 [==============================] - 739s - loss: 0.0158 - acc: 0.9961 - val_loss: 0.1836 - val_acc: 0.9670
Epoch 5/10
22667/22667 [==============================] - 739s - loss: 0.0127 - acc: 0.9971 - val_loss: 0.1860 - val_acc: 0.9658
Epoch 6/10
22667/22667 [==============================] - 739s - loss: 0.0107 - acc: 0.9974 - val_loss: 0.1880 - val_acc: 0.9685
Epoch 7/10
22667/22667 [==============================] - 739s - loss: 0.0104 - acc: 0.9978 - val_loss: 0.1921 - val_acc: 0.9668
Epoch 8/10
22667/22667 [========================

In [31]:
json_string = model.to_json()
open('models/CNN-H1_epoch-20.json', 'w').write(json_string)
model.save_weights('models/CNN-H1_epoch-20.h5', overwrite=True)